## Alternative Model (Preprocessing)
You need to run this before you run notebook 4b.

The wavs in your dataset will be converted to 9bit linear and 80-band mels.

In [ ]:
cd /home/erogol/Projects/WaveRNN/

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import librosa
import matplotlib.pyplot as plt
import math, pickle, os, glob
import numpy as np
from tqdm import tqdm_notebook as tqdm
from utils import *
from utils.display import *
from utils.generic_utils import load_config
from utils.audio import AudioProcessor
import IPython.display as ipd
from multiprocessing import Pool

In [ ]:
bits = 9
notebook_name = 'wavernn_ljspeech_ref'
config_path = '../TTS/config.json'

CONFIG = load_config(config_path)
ap = AudioProcessor(CONFIG.sample_rate, CONFIG.num_mels, CONFIG.min_level_db,
                    CONFIG.frame_shift_ms, CONFIG.frame_length_ms,
                    CONFIG.ref_level_db, CONFIG.num_freq, CONFIG.power, 0,
                    griffin_lim_iters=50)   

In [ ]:
# Point SEG_PATH to a folder containing your training wavs 
# Doesn't matter if it's LJspeech, CMU Arctic etc. it should work fine
SEG_PATH = '/home/erogol/Data/LJSpeech-1.1/wavs/' 
DATA_PATH = f'data/{notebook_name}/'
%mkdir -p $DATA_PATH

In [ ]:
def get_files(path, extension='.wav') :
    filenames = []
    for filename in glob.iglob(f'{path}/**/*{extension}', recursive=True):
        filenames += [filename]
    return filenames

In [ ]:
wav_files = get_files(SEG_PATH)
print(len(wav_files))

In [ ]:
def convert_file(path) :
    wav = ap.load_wav(path, encode=False)
    mel = ap.melspectrogram(wav)
    quant = (wav + 1.) * (2**bits - 1) / 2
    quant = quant.clip(0,2**bits-1)
    return mel.astype(np.float32), quant.astype(np.int), wav

# def convert_file(path) :
#     wav = ap.load_wav(path)
#     quant = ap.mulaw_encode(wav, 2**bits)
#     mel = ap.melspectrogram(wav)
#     return mel.astype(np.float32), quant.astype(np.int)

In [ ]:
wav_file = "/home/erogol/Data/LJSpeech-1.1/wavs/LJ018-0016.wav"
m, x, wav = convert_file(wav_file)

In [ ]:
print(x.max())
print(x.min())

In [ ]:
plot_spec(m)

In [ ]:
plot(x)

In [ ]:
x = 2 * x / (2**bits - 1) - 1

In [ ]:
# x_decoder = ap.mulaw_decode(x, 2**bits)
librosa.output.write_wav(DATA_PATH + 'test_quant.wav', x, sr=CONFIG.sample_rate)
ipd.Audio(x, rate=CONFIG.sample_rate)

In [ ]:
ipd.Audio(wav_files[1]) # load a local WAV file

In [ ]:
QUANT_PATH = DATA_PATH + 'quant/'
MEL_PATH = DATA_PATH + 'mel/'
%mkdir -p $QUANT_PATH
%mkdir -p $MEL_PATH

In [ ]:
wav_files[0].split('/')[-1][:-4]

In [ ]:
def process_wav(wav_path):
    idx = wav_path.split('/')[-1][:-4]
    m, x, wav = convert_file(wav_path)
    assert x.max() < 2**bits, wav_path
    assert x.min() >= 0
    np.save(f'{MEL_PATH}{idx}.npy', m)
    np.save(f'{QUANT_PATH}{idx}.npy', x)
    return idx

In [ ]:
# This will take a while depending on size of dataset
with Pool(8) as p:
    dataset_ids = list(tqdm(p.imap(process_wav, wav_files), total=len(wav_files)))

In [ ]:
with open(DATA_PATH + 'dataset_ids.pkl', 'wb') as f:
    pickle.dump(dataset_ids, f)